In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np

In [2]:
ratings = tfds.load("movielens/100k-ratings", split = "train")
movies = tfds.load("movielens/100k-movies", split = "train")

2022-09-27 21:23:14.821683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
ratings = ratings.map(lambda x : {
    "user_id" : x["user_id"],
    "movie_title" : x["movie_title"],
    "user_rating" : x["user_rating"]  
})

In [6]:
tf.random.set_seed(42)

shuffled = ratings.shuffle(100000, seed = 42, reshuffle_each_iteration=False) # forgot to put the amount of shuffle times

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

user_ids = ratings.batch(1000000).map(lambda x: x["user_id"])
movie_titles = ratings.batch(1000000).map(lambda x: x["movie_title"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [38]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimensions = 32

    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimensions)
    ])

    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimensions)
    ])

    self.ratings = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1)])

    def call(self, inputs):

            user_id, movie_title = inputs

            user_embedding = self.user_embeddings(user_id)
            movie_embedding = self.movie_embeddings(movie_title)

            return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))


In [39]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

NotImplementedError: Exception encountered when calling layer "ranking_model_14" (type RankingModel).

Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.

Call arguments received by layer "ranking_model_14" (type RankingModel):
  • inputs=(["'42'"], ['"One Flew Over the Cuckoo\'s Nest (1975)"'])
  • training=None
  • mask=None